In [52]:
import os
from dotenv import load_dotenv, find_dotenv
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

In [53]:
# get environment variables
load_dotenv(find_dotenv(), override=True)

True

In [54]:
# setup Dune Python client
dune = DuneClient.from_env()

# Database connection URL
db_url = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(db_url)

In [46]:
# Get latest price from database
query = "SELECT max(block_time_unix) FROM dune_data;"
last_swap = pd.read_sql(query, engine)

In [47]:
last_swap = last_swap['max'][0]


In [34]:
# set the dune query id:
dune_query_id = 4002884

In [69]:
"""  query the query (execute and get latest result) """
query = QueryBase(
    query_id=dune_query_id,

    params=[
        QueryParameter.number_type(name="day_limit", value=1704067200), # default is 1704067200 or 2024-07-01
    ],
)

query_result = dune.run_query_dataframe(
  query=query
  # , ping_frequency = 10 # uncomment to change the seconds between checking execution status, default is 1 second
  # , performance="large" # uncomment to run query on large engine, default is medium
  # , batch_size = 5_000 # uncomment to change the maximum number of rows to retrieve per batch of results, default is 32_000
) 

2024-08-21 14:18:57,494 INFO dune_client.api.base executing 4002884 on medium cluster
2024-08-21 14:18:57,707 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.PENDING (queue position: 1)
2024-08-21 14:18:58,762 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.EXECUTING
2024-08-21 14:18:59,819 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.EXECUTING
2024-08-21 14:19:00,886 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.EXECUTING
2024-08-21 14:19:01,947 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.EXECUTING
2024-08-21 14:19:03,010 INFO dune_client.api.base waiting for query execution 01J5TE0EF9K2G9199YCF58ZZKD to complete: ExecutionState.EXECUTING
2024-08-21 14:19:04,073 INFO dune_clie

In [70]:
# Remove ' UTC' from the datetime strings
query_result['block_time'] = pd.to_datetime(query_result['block_time'].str.replace(' UTC', ''))

# Convert the datetime objects to Unix timestamps (seconds since epoch)
query_result['block_time_unix'] = query_result['block_time'].astype(int) // 10**9

In [71]:
# add sell_price
query_result['sell_price'] = query_result['dst_tokens'] / query_result['src_tokens']

In [72]:
# """ get the latest executed result without triggering a new execution """
# # query_result = dune.get_latest_result(dune_query_id) # get latest result in json format
# query_result = dune.get_latest_result_dataframe(dune_query_id) # get latest result in Pandas dataframe format
# query_result.shape

In [73]:
# Save to PostgreSQL

# Use SQLAlchemy to create the table in a specified schema
query_result.to_sql('dune_data', engine, schema='public', if_exists='replace', index=False)

483

In [74]:
# Query the full table

query = "SELECT * FROM dune_data;"

df = pd.read_sql(query, engine)

In [75]:
df

,block_time,tx_hash,src_token_address,src_token_symbol,src_tokens,dst_token_address,dst_token_symbol,dst_tokens,block_time_unix,sell_price
0,2024-06-24 20:10:23,0x265338185467fa750d52375be392c56df75f93bea0f1...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,129.413771,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,428328.403249,1719259823,3309.759082
1,2024-06-17 13:20:35,0xe2472158b2643d02b6c80f6a86c97f293b3268adce0a...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,1.134331,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,4006.302981,1718630435,3531.863147
2,2024-06-24 01:25:59,0xb255d308279f53ed161791085c30113fff7ff48f8561...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,1.112803,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,3802.479134,1719192359,3417.028872
3,2024-06-17 02:08:35,0xdf7fd8e04989d59a824a93e1226299addd1d56a9bd13...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.458980,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,1652.467808,1718590115,3600.304229
4,2024-06-19 20:27:23,0xce40c615a3a3425614db1bb2a22c0463dd268b854247...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,0.518160,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,1827.579803,1718828843,3527.058892
...,...,...,...,...,...,...,...,...,...,...
478,2024-06-29 04:31:11,0xc814b0ac44aab39af0484e2e6356f64bb2b949b439dc...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,10.172217,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,34353.812358,1719635471,3377.219788
479,2024-06-19 15:05:47,0xb2c772b9ffca86b41a4639458a7c0dd23d31a0f7b57c...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,1.170940,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,4117.394649,1718809547,3516.314229
480,2024-06-27 14:20:59,0x2f9f1ad63719e289a29b47a94a3bf219148f1ced8bad...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,25.560105,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,88529.651253,1719498059,3463.587214
481,2024-06-27 14:54:11,0x73c422c7ea886d1f98e8f9c24140d10404365ae3be01...,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,14.745112,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,50807.480792,1719500051,3445.716885
